In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import automutualinformation
import pickle

# Load The Sequences!
with open('symbolic_sequences_pup2mother_All.bkp.pkl','rb') as f:
    symbolic_sequences = pickle.load(f)

In [5]:
# wrapper lib
from avgn.utils.paths import DATA_DIR, ensure_dir
from pathlib2 import Path


DATA_DIR = Path('./data')
DSLOC = DATA_DIR / 'raw' / 'fruitbat'

PROJECT_DIR set on:  /data0/home/h21/luas6629/Thesis


In [6]:
DSLOC

PosixPath('data/raw/fruitbat')

In [7]:
import torchaudio
waveform, sample_rate = torchaudio.load( DSLOC / 'zip_contents'/ 'files101' /  '120601000005102988.WAV'   )

In [8]:
symbolic_sequences['Path'] =  './data/raw/fruitbat/zip_contents/' + symbolic_sequences['Folder'] +'/' + symbolic_sequences['File name']

In [28]:
symbolic_sequences['waveform'] = symbolic_sequences['Path'].apply(lambda x  : torchaudio.load(x)[0])

In [33]:
symbolic_sequences.head()

,File name,Seq_Syllables,Duration,Folder,labels.label_addressee,labels.label_context,Path,waveform_col
0,121002212651151905.WAV,"[8, 2, 9]","[28341, 19373, 28746]",files201,[209],[6],./data/raw/fruitbat/zip_contents/files201/1210...,"[[tensor(-0.0002), tensor(-0.0005), tensor(0.0..."
1,121003075445185924.WAV,[9],[26746],files201,[209],[6],./data/raw/fruitbat/zip_contents/files201/1210...,"[[tensor(3.0518e-05), tensor(-3.0518e-05), ten..."
2,121004173718193940.WAV,"[9, 9, 9]","[26773, 27746, 25246]",files201,[209],[6],./data/raw/fruitbat/zip_contents/files201/1210...,"[[tensor(-0.0008), tensor(0.0008), tensor(0.00..."
3,121005221219635013.WAV,"[7, 9, 9]","[26756, 36746, 39246]",files202,[209],[6],./data/raw/fruitbat/zip_contents/files202/1210...,"[[tensor(-3.0518e-05), tensor(-0.0003), tensor..."
4,121006093630175196.WAV,[9],[28746],files202,[209],[6],./data/raw/fruitbat/zip_contents/files202/1210...,"[[tensor(0.0003), tensor(-0.0006), tensor(-0.0..."


In [9]:
import torch
from torch.utils.data import Dataset

class WaveformDataset(Dataset):
    def __init__(self, dataframe, waveform_col, label_col, transform=None):
        """
        Custom PyTorch dataset for waveform data with multiple labels.
        
        Args:
            dataframe (pandas.DataFrame): Input dataframe that contains waveform data and labels.
            waveform_col (str): Name of the column in the dataframe that contains waveform data.
            label_col (str): Name of the column in the dataframe that contains label data.
            transform (callable, optional): Optional data transformation function to be applied to the waveforms.
        """
        self.dataframe = dataframe
        self.waveform_col = waveform_col
        self.label_col = label_col
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        waveform = self.dataframe.loc[index, self.waveform_col]  # Get waveform data at index
        label = self.dataframe.loc[index, self.label_col]  # Get label data at index

        # Apply data transformation if provided
        if self.transform is not None:
            waveform = self.transform(waveform)

        return waveform, label

In [12]:
class AvesClassifier(nn.Module):
    def __init__(self, model_path, num_classes, embeddings_dim=768, multi_label=False):

        super().__init__()

        models, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([model_path])
        self.model = models[0]
        self.model.feature_extractor.requires_grad_(False)
        self.head = nn.Linear(in_features=embeddings_dim, out_features=num_classes)

        if multi_label:
            self.loss_func = nn.BCEWithLogitsLoss()
        else:
            self.loss_func = nn.CrossEntropyLoss()

    def forward(self, x, y=None):
        out = self.model.extract_features(x)[0]
        out = out.mean(dim=1)             # mean pooling
        logits = self.head(out)

        loss = None
        if y is not None:
            loss = self.loss_func(logits, y)

        return loss, logits

In [44]:
# Instantiate WaveformDataset
dataset = WaveformDataset(symbolic_sequences, 'waveform_col','labels.label_context')

In [13]:
import fairseq


In [71]:
dataset.__getitem__(0)

(tensor([[-0.0002, -0.0005,  0.0008,  ...,  0.0010,  0.0003,  0.0003]]), ['6'])

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam

In [48]:
# Split dataset into train and test sets
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


In [49]:
# Define data loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [25]:
!pip uninstall fairseq -y

Found existing installation: fairseq 0.12.2
Uninstalling fairseq-0.12.2:
  Successfully uninstalled fairseq-0.12.2


In [26]:
pip install fairseq --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 54.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11162107 sha256=2d340ee640a5a1f1d2daed98eb5a17e63ebcb5fcefbaed57760182b05025f699
  Stored in directory: /tmp/pip-ephem-wheel-cache-j6nhxkhz/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
Successfully built fairseq
Note: you may need to restart the kernel to use updated packages.


In [23]:
!pip3 cache list

Cache contents:

 - antlr4_python3_runtime-4.8-py3-none-any.whl (141 kB)
 - datashape-0.5.2-py3-none-any.whl (59 kB)
 - fairseq-0.12.2-cp310-cp310-linux_x86_64.whl (11.2 MB)
 - fitter-1.5.2-py3-none-any.whl (25 kB)
 - future-0.18.3-py3-none-any.whl (492 kB)
 - hdbscan-0.8.29-cp310-cp310-linux_x86_64.whl (3.0 MB)
 - pathtools-0.1.2-py3-none-any.whl (8.8 kB)
 - pomegranate-0.14.8-cp310-cp310-linux_x86_64.whl (18.2 MB)
 - pyclustering-0.10.1.2-py3-none-any.whl (2.4 MB)
 - pynndescent-0.5.8-py3-none-any.whl (55 kB)
 - tinytag-1.8.1-py3-none-any.whl (36 kB)
 - umap_learn-0.5.3-py3-none-any.whl (82 kB)


In [28]:
from fairseq import metrics

ImportError: cannot import name 'metrics' from 'fairseq' (unknown location)

In [30]:
import fairseq.checkpoint_utils

ImportError: cannot import name 'metrics' from 'fairseq' (unknown location)

In [29]:
# Instantiate AvesClassifier
model = AvesClassifier(
    model_path='./aves-base-bio.pt',
    num_classes=10)


AttributeError: module 'fairseq' has no attribute 'checkpoint_utils'

In [52]:
# Define loss function and optimizer
criterion = model.loss_func
optimizer = Adam(model.parameters(), lr=0.001)

NameError: name 'model' is not defined